# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [32]:
import pandas as pd
import numpy as np
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)

# Não há duplicados

(27028, 69)


In [9]:
# 2) seu código aqui
sinasc.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [10]:
# 3) seu código aqui
sinasc.drop_duplicates().shape

(27028, 69)

In [15]:
# 4) seu código aqui
sinasc[sinasc.duplicated()]

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea


In [16]:
# 5) seu código aqui
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [43]:
# 9) seu código aqui
sinasc = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]

In [44]:
sinasc

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [45]:
sinasc.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [46]:
sinasc[sinasc['APGAR5'].isnull()]

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
558,1,30,1.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,NaN
2009,3,35,5.0,1 a 3 anos,6.0,32 a 36 semanas,Única,3,NaN
2340,1,29,NaN,8 a 11 anos,2.0,37 a 41 semanas,Única,4,NaN
2704,3,13,1.0,4 a 7 anos,1.0,32 a 36 semanas,Única,2,NaN
2705,3,28,1.0,8 a 11 anos,4.0,32 a 36 semanas,Única,3,NaN
...,...,...,...,...,...,...,...,...,...
25378,3,20,1.0,12 anos ou mais,0.0,NaN,Única,1,NaN
25864,3,18,5.0,8 a 11 anos,0.0,32 a 36 semanas,Única,3,NaN
26715,1,24,5.0,8 a 11 anos,3.0,37 a 41 semanas,Única,2,NaN
26923,1,20,5.0,8 a 11 anos,0.0,32 a 36 semanas,Única,3,NaN


In [48]:
sinasc['APGAR5'].isnull().sum()

103

In [61]:
sinasc['APGAR5'].dropna()

0        10.0
1         9.0
2        10.0
3        10.0
4        10.0
         ... 
27023     9.0
27024     9.0
27025    10.0
27026     9.0
27027     8.0
Name: APGAR5, Length: 23721, dtype: float64

In [65]:
sinasc['APGAR5'].isnull().sum()

0

In [67]:
print(sinasc[['ESTCIVMAE', 'CONSULTAS']])

       ESTCIVMAE  CONSULTAS
0            5.0          4
1            2.0          4
2            9.0          4
3            5.0          3
4            2.0          4
...          ...        ...
27023        2.0          4
27024        1.0          4
27025        2.0          4
27026        2.0          4
27027        5.0          4

[23721 rows x 2 columns]


In [87]:
sinasc = sinasc[(sinasc['CONSULTAS'] != 9.0) & (sinasc['ESTCIVMAE'] != 9.0)]

In [83]:
sinasc

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [ ]:
QTDFILVIVO

In [90]:
sinasc.isnull().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

Resolvi remover os dados que faltantes pois, se considermos o universo em relação aos elementos "nulos", não impactaria na análise final dos dados!!

Entre 8 e 10 está em uma faixa 'normal'.
Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
Entre 4 e 5 significa 'asfixia moderada'.
Entre 0 e 3 significa 'asfixia severa'.

In [93]:
sinasc.loc[sinasc['APGAR5'] <= 3, 'Ind_Asfixia'] = 'Asfixia Severa'
sinasc.loc[(sinasc['APGAR5'] > 3) & (sinasc['APGAR5'] <= 5),
           'Ind_Asfixia'] = 'Asfixia Moderada'
sinasc.loc[(sinasc['APGAR5'] > 5) & (sinasc['APGAR5'] <= 7),
           'Ind_Asfixia'] = 'Asfixia Leve'
sinasc.loc[(sinasc['APGAR5'] > 7), 'Ind_Asfixia'] = 'Normal'

In [94]:
sinasc

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,Ind_Asfixia
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,Normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,Normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,Normal


In [97]:
sinasc['APGAR5'].quantile([0.25,0.5,0.75])

0.25     9.0
0.50     9.0
0.75    10.0
Name: APGAR5, dtype: float64

In [98]:
quartil_1, quartil_2, quartil_3 = sinasc['APGAR5'].quantile([0.25,0.5,0.75])

In [99]:
sinasc['Quartil'] = np.where(
    sinasc['APGAR5'] < 9.0, '1o_quartil',
    np.where(sinasc['APGAR5'] < 9.0, '2o_quartil',
             np.where(sinasc['APGAR5'] < 10.0, '3o_quartil', '4o_quartil')))

In [100]:
sinasc

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,Ind_Asfixia,Quartil
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal,4o_quartil
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Normal,3o_quartil
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Normal,4o_quartil
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal,4o_quartil
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal,4o_quartil
...,...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,Normal,3o_quartil
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,Normal,3o_quartil
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal,4o_quartil
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,Normal,3o_quartil


In [102]:
dummies = pd.get_dummies(sinasc['Ind_Asfixia'])

In [111]:
sinasc = pd.concat([sinasc, dummies], axis=1)

In [112]:
sinasc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23576 entries, 0 to 27027
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   LOCNASC           23576 non-null  int64  
 1   IDADEMAE          23576 non-null  int64  
 2   ESTCIVMAE         23576 non-null  float64
 3   ESCMAE            23576 non-null  object 
 4   QTDFILVIVO        23576 non-null  float64
 5   GESTACAO          23576 non-null  object 
 6   GRAVIDEZ          23576 non-null  object 
 7   CONSULTAS         23576 non-null  int64  
 8   APGAR5            23576 non-null  float64
 9   Ind_Asfixia       23576 non-null  object 
 10  Quartil           23576 non-null  object 
 11  Asfixia Leve      23576 non-null  uint8  
 12  Asfixia Moderada  23576 non-null  uint8  
 13  Asfixia Severa    23576 non-null  uint8  
 14  Normal            23576 non-null  uint8  
dtypes: float64(3), int64(3), object(5), uint8(4)
memory usage: 2.2+ MB


In [114]:
percentage = (sinasc['Normal'].sum() / len(sinasc['Normal'])) * 100
percentage

98.29487614523244

In [115]:
percentage = (sinasc['Asfixia Severa'].sum() / len(sinasc['Asfixia Severa'])) * 100
percentage

0.2672209026128266

In [116]:
percentage = (sinasc['Asfixia Moderada'].sum() / len(sinasc['Asfixia Moderada'])) * 100
percentage

0.2290464879538514

In [117]:
percentage = (sinasc['Asfixia Leve'].sum() / len(sinasc['Asfixia Leve'])) * 100
percentage

1.2088564642008823

In [119]:
sinasc.columns

Index(['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO',
       'GRAVIDEZ', 'CONSULTAS', 'APGAR5', 'Ind_Asfixia', 'Quartil',
       'Asfixia Leve', 'Asfixia Moderada', 'Asfixia Severa', 'Normal'],
      dtype='object')

In [121]:
sinasc.columns = ['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo', 'gestacao', 'gravidez', 'consultas', 'apgar5', 'ind_asfixia', 'quartil', 'asfixia_leve', 'asfixia_moderada', 'asfixia_severa', 'normal']

In [122]:
sinasc

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar5,ind_asfixia,quartil,asfixia_leve,asfixia_moderada,asfixia_severa,normal
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal,4o_quartil,0,0,0,1
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Normal,3o_quartil,0,0,0,1
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Normal,4o_quartil,0,0,0,1
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal,4o_quartil,0,0,0,1
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal,4o_quartil,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,Normal,3o_quartil,0,0,0,1
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,Normal,3o_quartil,0,0,0,1
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal,4o_quartil,0,0,0,1
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,Normal,3o_quartil,0,0,0,1
